# A Reduction Tale

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to need reductions a lot and we want to choose the best container for performing them.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.031 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 35.035 MiB


and choose a different container for the data that we want to reduce, starting with a list:

## Regular lists

In [2]:
a = [float(i) for i in range(10*1000*1000)]

In [2] used 545.148 MiB RAM in 2.220s, peaked 34.074 MiB above current, total RAM usage 580.184 MiB


Now, proceed with a simple reduction (sum):

In [3]:
t = %timeit -o sum(a)

10 loops, best of 3: 73.1 ms per loop
In [3] used 0.051 MiB RAM in 2.983s, peaked 0.000 MiB above current, total RAM usage 580.234 MiB


which, in MFLOPS (Mega-FloatingPointOps-Per-Second) is:

In [4]:
print("MFLOPS:", round((len(a) / t.best / 1e6), 1))

('MFLOPS:', 136.8)
In [4] used 0.012 MiB RAM in 0.079s, peaked 0.000 MiB above current, total RAM usage 580.246 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not the best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

## NumPy

In [10]:
import numpy as np

In [10] used 9.641 MiB RAM in 0.044s, peaked 0.000 MiB above current, total RAM usage 590.254 MiB


In [35]:
na = np.array(a, dtype=np.float64)

In [35] used 0.000 MiB RAM in 0.318s, peaked 0.000 MiB above current, total RAM usage 699.539 MiB


In [32]:
print("SIZE:", round((na.size * na.itemsize) / 2**20., 3))

('SIZE:', 76.294)
In [32] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 699.539 MiB


We see that, with 8 bytes/element, NumPy is a very efficient container.

In [36]:
t = %timeit -o sum(na)

1 loop, best of 3: 1.07 s per loop
In [36] used 0.000 MiB RAM in 4.294s, peaked 0.000 MiB above current, total RAM usage 699.539 MiB


In [37]:
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

('MFLOPS:', 9.339)
In [37] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 699.539 MiB


### Exercise

The performance for NumPy is several times slower than the computation with the list.  Why so?

*Hint: * We are using sum() which is a Python function.

### Solution

NumPy has a lot of overhead in producing a Python integer for every element in the array for feeding it to the sum().

*Hint:* Use internal NumPy methods (ufuncs) when possible.

In [38]:
t = %timeit -o na.sum()

100 loops, best of 3: 7.05 ms per loop
In [38] used 0.000 MiB RAM in 2.914s, peaked 0.000 MiB above current, total RAM usage 699.539 MiB


In [39]:
print("FLOPS:", round(len(a) / t.best / 1e6, 3))

('FLOPS:', 1419.267)
In [39] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 699.539 MiB


This is about 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.

## Exercise

The speed in the above reduction is limited by memory speed, not CPU speed.  Could you provide a hint on the maximum memory speed that supports your laptop?

#### Solution

In [16]:
# This is an easy one.  Just divide the size of the dataset by the time that takes the reduction
(na.size * na.itemsize) / t.best / 2**30

13.287169101655559

In [16] used 0.008 MiB RAM in 0.086s, peaked 0.000 MiB above current, total RAM usage 590.441 MiB


So, in this case the memory bandwidth is close to 10 GB/s.

## Using compressed in-memory containers with bcolz

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

In [17]:
import bcolz
bcolz.print_versions()
bcolz.defaults.cparams['cname'] = 'blosclz'
bcolz.defaults.cparams['clevel'] = 9
bcolz.defaults.cparams['shuffle'] = bcolz.SHUFFLE
bcolz.set_nthreads(4)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
bcolz version:     1.0.0rc2
NumPy version:     1.10.4
Blosc version:     1.8.0 ($Date:: 2016-03-31 #$)
Blosc compressors: ['blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib']
Numexpr version:   2.5.1
Python version:    2.7.11 |Continuum Analytics, Inc.| (default, Dec  6 2015, 18:08:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Platform:          linux2-x86_64
Byte-ordering:     little
Detected cores:    4
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


4

In [17] used 20.375 MiB RAM in 0.216s, peaked 0.000 MiB above current, total RAM usage 610.816 MiB


In [18]:
ca = bcolz.carray(na)

In [18] used 0.289 MiB RAM in 0.029s, peaked 0.000 MiB above current, total RAM usage 611.105 MiB


In [19]:
print("mem_used:", mw.measurements.memory_delta)

('mem_used:', 0.2890625)
In [19] used 2.008 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 613.113 MiB


Ok, this time the amount of memory used seems much lower.  Also, bcolz containers can provide an estimation on how much memory they are taking; let's have a look:

In [20]:
ca

carray((10000000,), float64)
  nbytes: 76.29 MB; cbytes: 1.01 MB; ratio: 75.31
  cparams := cparams(clevel=9, shuffle=1, cname='blosclz')
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [20] used -1.953 MiB RAM in 0.004s, peaked 1.965 MiB above current, total RAM usage 611.160 MiB


In this case we see that bcolz estimation is reasonably close to `ipython_memwatcher` measurements.  Let's have a look at the speed of the reduction:

In [21]:
t = %timeit -o ca.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

10 loops, best of 3: 20.7 ms per loop
('MFLOPS:', 483.458)
In [21] used 0.020 MiB RAM in 0.880s, peaked 0.000 MiB above current, total RAM usage 611.180 MiB


This is around 2~5x slower (depending on the machine) than a regular NumPy array, but the size of the array is an impressive 75x smaller.  But is compression the only responsible of the overhead?  Let's investigate a bit further.

## Using uncompressed containers with bcolz

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [22]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [22] used 9.555 MiB RAM in 0.369s, peaked 0.000 MiB above current, total RAM usage 620.734 MiB


In [23]:
cau

carray((10000000,), float64)
  nbytes: 76.29 MB; cbytes: 76.52 MB; ratio: 1.00
  cparams := cparams(clevel=0, shuffle=1, cname='blosclz')
[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   9.99999700e+06
   9.99999800e+06   9.99999900e+06]

In [23] used -0.051 MiB RAM in 0.051s, peaked 0.000 MiB above current, total RAM usage 620.684 MiB


In [24]:
t = %timeit -o cau.sum()
print("MFLOPS:", round(len(a) / t.best / 1e6, 3))

100 loops, best of 3: 17.5 ms per loop
('MFLOPS:', 570.666)
In [24] used 0.008 MiB RAM in 7.292s, peaked 0.000 MiB above current, total RAM usage 620.691 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one, so compressing is not the only source of the overhead (and it is very minor in fact).  The actual source of the difference is the memory layout of the different containers (bcolz layout is a bit more complex than NumPy).

So, bcolz allows to use compressed in-memory data containers at the cost of some performance (compared with NumPy).  But the performance overhead is rather small, and sometimes you prefer to keep more data in-memory.

On another hand, in the next notebooks we are going to see that bcolz can be pretty close to NumPy, performance wise, in some scenarios.